<a href="https://colab.research.google.com/github/kategavrishina/info-search/blob/master/HW4/hw4_semantics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install gensim --upgrade

     |████████████████████████████████| 24.2MB 161kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
! pip install natasha

     |████████████████████████████████| 34.4MB 118kB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 8.2MB 11.4MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26100 sha256=98a85534b58c31baf9d5d9d6493aafacc663b800373c9e9625431d82d786f62f
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


Реализуйте поиск по нашему стандартному Covid корпусу с помощью модели на Araneum двумя способами:

    1. преобразуйте каждый документ в вектор через усреднение векторов его слов и реализуйте поисковик как 
    обычно через умножение матрицы документов коллекции на вектор запроса 
    2. экспериментальный способ - реализуйте поиск ближайшего документа в коллекции к запросу, преобразовав 
    каждый документ в матрицу (количество слов x размер модели)
    
Посчитайте качество поиска для каждой модели на тех же данных, что и в предыдущем задании. В качестве препроцессинга используйте две версии - с удалением NER и без удаления.  

In [3]:
%load_ext autoreload

from gensim.models import KeyedVectors

In [4]:
import urllib

In [5]:
urllib.request.urlretrieve("https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz", "araneum_none_fasttextcbow_300_5_2018.tgz")

('araneum_none_fasttextcbow_300_5_2018.tgz',
 <http.client.HTTPMessage at 0x7fdcc07dc978>)

In [6]:
! tar xvfz araneum_none_fasttextcbow_300_5_2018.tgz

araneum_none_fasttextcbow_300_5_2018.model
araneum_none_fasttextcbow_300_5_2018.model.vectors_ngrams.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors_vocab.npy


In [7]:
model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)

In [8]:
! pip install pymorphy2

In [9]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

In [12]:
import re

In [13]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()

morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [14]:
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [15]:
def preprocess(text):
    reg = re.compile('[^а-яА-Я ]')
    text = reg.sub('', text.lower().strip())
    words = text.strip().split()
    s = []
    for word in words:
        if word != '':
            word = morph.parse(word)[0].normal_form
            if word not in russian_stopwords:
                s.append(word)
    return s

In [16]:
def preprocess_with_natasha(text: str):
    extractors = [
              dates_extractor,
              money_extractor,
              addr_extractor
    ]
    
    doc = Doc(text)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    indices = []
    for span in doc.spans:
        if span.type == "PER":
            indices.append((span.start, span.stop))
    for pair in indices[::-1]:
        text = text[:pair[0]] + text[pair[1]:]

    for extractor in extractors:
        idx = []
        for match in extractor(text):
            idx.append((match.start, match.stop))
        for pair in idx[::-1]:
            text = text[:pair[0]] + text[pair[1]:]
    
    return preprocess(text)

In [17]:
import numpy as np

In [18]:
import pandas as pd

In [19]:
ans = pd.read_excel('answers_base.xlsx')

In [20]:
answers = pd.DataFrame(ans[['Номер связки', 'Текст ответа']])

In [21]:
ans['Вопросы'] = ans['Текст вопросов'].apply(lambda text: text.split('\n'))

In [22]:
ans = ans.explode('Вопросы')

In [23]:
ans.reset_index(drop=True, inplace=True)
ans.drop(['Текст вопросов', 'Текст ответа', 'Тематика'], axis=1, inplace=True)

In [24]:
ans.head()

,Номер связки,Вопросы
0,57,У ребенка в школе продлили каникулы. Могу ли я...
1,57,Больничный лист?
2,57,"Есть ли компенсация, в случае если есть разниц..."
3,57,как оплачивается больничный при коронавирусе?
4,57,"Я контактный, дадут ли больничный?"


In [25]:
q = pd.read_excel('queries_base.xlsx', usecols='A, B', names=['Вопросы', 'Номер связки'])

In [26]:
q = q[['Номер связки', 'Вопросы']]

In [27]:
data = pd.concat([ans, q], ignore_index=True)

In [28]:
from tqdm.auto import tqdm

tqdm.pandas()

In [29]:
data['Preprocessed'] = data['Вопросы'].progress_apply(lambda text: ' '.join(preprocess(str(text))))

In [30]:
data['Natasha'] = data['Вопросы'].progress_apply(lambda text: ' '.join(preprocess_with_natasha(str(text))))

In [31]:
data.head()

,Номер связки,Вопросы,Preprocessed,Natasha
0,57.0,У ребенка в школе продлили каникулы. Могу ли я...,ребёнок школа продлить каникулы мочь взять бол...,ребёнок школа продлить каникулы мочь взять бол...
1,57.0,Больничный лист?,больничный лист,больничный лист
2,57.0,"Есть ли компенсация, в случае если есть разниц...",компенсация случай разница оплата больничный з...,компенсация случай разница оплата больничный з...
3,57.0,как оплачивается больничный при коронавирусе?,оплачиваться больничный коронавирус,оплачиваться больничный коронавирус
4,57.0,"Я контактный, дадут ли больничный?",контактный дать больничный,контактный дать больничный


In [78]:
data.drop(data[data['Номер связки'].isna()].index, inplace=True)

In [81]:
data.reset_index(inplace=True)

In [82]:
data.to_excel('covid_corpus.xlsx')

In [83]:
data_train, data_test = data[:int(len(data)*0.7)], data[int(len(data)*0.7):]

In [84]:
def normalize_vec(v):
     return v / np.sqrt(np.sum(v ** 2))

In [91]:
def get_vector_mean(lemmas):
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        try:
            lemmas_vectors[idx] = model[lemma]
        except AttributeError:
            continue

    if lemmas_vectors.shape[0] is not 0:
        vec = np.mean(lemmas_vectors, axis=0)

    if np.sum(vec ** 2) != 0:
        vec = normalize_vec(vec)
    
    return vec

In [92]:
matrix = np.zeros((len(data), model.vector_size))
for idx, text in enumerate(data.Preprocessed):
    matrix[idx] = get_vector_mean(text.split())

In [93]:
matrix_natasha = np.zeros((len(data), model.vector_size))
for idx, text in enumerate(data.Natasha):
    matrix_natasha[idx] = get_vector_mean(text.split())

In [94]:
matrix_train, matrix_test = matrix[:int(len(data)*0.7)], matrix[int(len(data)*0.7):]
matrix_train_n, matrix_test_n = matrix_natasha[:int(len(data)*0.7)], matrix_natasha[int(len(data)*0.7):]

In [95]:
def get_classdic_answer(vec, matrix):
    res = normalize_vec(vec).dot(matrix.T)
    return int(data_train.loc[res.argmax()]['Номер связки'])

In [97]:
data_test['classic'] = [get_classic_answer(i, matrix_train) for i in matrix_test]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
data_test['classic_natasha'] = [get_classic_answer(i, matrix_train_n) for i in matrix_test_n]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
data_test[data_test['Номер связки'] == data_test.classic].shape[0]/data_test.shape[0]

0.5206073752711496

In [100]:
data_test[data_test['Номер связки'] == data_test.classic_natasha].shape[0]/data_test.shape[0]

0.5184381778741866

In [205]:
def create_doc_matrix(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))

    for idx, lemma in enumerate(lemmas):
        lemmas_vectors[idx] = normalize_vec(model[lemma])
    
    if text == '':
        lemmas_vectors = np.zeros((1, model.vector_size))

    return lemmas_vectors  

In [206]:
list_of_mtx = []
for idx, text in enumerate(data.Preprocessed):
    list_of_mtx.append(create_doc_matrix(text))

list_of_mtx_n = []
for idx, text in enumerate(data.Natasha):
    list_of_mtx_n.append(create_doc_matrix(text))

In [207]:
data.Preprocessed[16]

''

In [208]:
list_of_mtx_n[16]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [209]:
list_of_mtx[16]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [210]:
list_train, list_test = list_of_mtx[:int(len(data)*0.7)], list_of_mtx[int(len(data)*0.7):]
list_train_n, list_test_n = list_of_mtx_n[:int(len(data)*0.7)], list_of_mtx_n[int(len(data)*0.7):]

In [211]:
list_test[5] == list_test_n[5]

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [212]:
data_test[:10]

,index,Номер связки,Вопросы,Preprocessed,Natasha,classic,classic_natasha,experiment,experiment_natasha
2149,2155,308.0,Добрый вечер. Слетал в Турцию . Сдал тест. Пол...,добрый вечер слетать турция сдать тест получит...,добрый вечер слетать турция сдать тест получит...,308,308,37.0,37.0
2150,2156,70.0,\nМне 60 лет и я почему-то помню урок 5-го кла...,год почемуть помнить урок го класс работать ды...,год почемуть помнить урок го класс работать ды...,6,6,37.0,37.0
2151,2157,6.0,"При въезде в Москву на а/м из регионов, без пр...",въезд москва ама регион признак заболевание тр...,въезд ама регион признак заболевание требовать...,308,257,37.0,37.0
2152,2158,308.0,"Добрый день!\n\nЯ гражданин России, живу в Мос...",добрый денья гражданин россия жить москва мать...,добрый денья гражданин жить мать гражданка хот...,308,308,37.0,37.0
2153,2159,308.0,"\nДобрый день! Подскажите, пожалуйста, тест на...",добрый день подсказать пожалуйста тест ковид в...,добрый день подсказать пожалуйста тест ковид в...,37,308,37.0,37.0
2154,2160,37.0,Здравствуйте.На следующей неделе отправляюсь в...,здравствуйесть следующий неделя отправляться к...,здравствуйесть следующий неделя отправляться к...,135,135,37.0,37.0
2155,2161,308.0,\nВы могли бы разъяснить правила въезда Америк...,мочь разъяснить правило въезд американский гра...,мочь разъяснить правило въезд американский гра...,308,308,37.0,37.0
2156,2162,308.0,Добрый день. Скажите пожалуйста нужно ли сидет...,добрый день сказать пожалуйста нужно сидеть ре...,добрый день сказать пожалуйста нужно сидеть ре...,308,308,37.0,37.0
2157,2163,308.0,Добрый день!\n\nВаша горячая линия работает уж...,добрый деньваша горячий линия работать ужаснон...,добрый деньваша горячий линия работать ужаснон...,37,37,37.0,37.0
2158,2164,308.0,Если я вернулся из-за границы и должен сдать т...,вернуться изз граница должный сдать тест корон...,вернуться изз граница должный сдать тест корон...,308,308,37.0,37.0


In [218]:
def search(docs, query, reduce_func=np.max, axis=0):
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    return np.argmax(sims)

In [219]:
data_test['experiment'] = [data_train.loc[search(list_train, mat)]['Номер связки'] for mat in tqdm(list_test)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [220]:
data_test['experiment_natasha'] = [data_train.loc[search(list_train_n, mat)]['Номер связки'] for mat in tqdm(list_test_n)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [221]:
data_test[data_test['Номер связки'] == data_test.experiment].shape[0]/data_test.shape[0]

0.29067245119305857

In [222]:
data_test[data_test['Номер связки'] == data_test.experiment_natasha].shape[0]/data_test.shape[0]

0.3156182212581345

In [223]:
data_test.head()

,index,Номер связки,Вопросы,Preprocessed,Natasha,classic,classic_natasha,experiment,experiment_natasha
2149,2155,308.0,Добрый вечер. Слетал в Турцию . Сдал тест. Пол...,добрый вечер слетать турция сдать тест получит...,добрый вечер слетать турция сдать тест получит...,308,308,308.0,308.0
2150,2156,70.0,\nМне 60 лет и я почему-то помню урок 5-го кла...,год почемуть помнить урок го класс работать ды...,год почемуть помнить урок го класс работать ды...,6,6,37.0,37.0
2151,2157,6.0,"При въезде в Москву на а/м из регионов, без пр...",въезд москва ама регион признак заболевание тр...,въезд ама регион признак заболевание требовать...,308,257,37.0,37.0
2152,2158,308.0,"Добрый день!\n\nЯ гражданин России, живу в Мос...",добрый денья гражданин россия жить москва мать...,добрый денья гражданин жить мать гражданка хот...,308,308,37.0,37.0
2153,2159,308.0,"\nДобрый день! Подскажите, пожалуйста, тест на...",добрый день подсказать пожалуйста тест ковид в...,добрый день подсказать пожалуйста тест ковид в...,37,308,6.0,308.0
